In [1]:
import pandas as pd
import matplotlib as plt
import os 
import folium
import geopandas as gpd
import warnings
warnings.filterwarnings('ignore')

In [2]:
##set dir here

In [3]:
# Year:             2014-2018
# Geographic level: 5-Digit ZIP Code Tabulation Area

# Dataset:          2018 American Community Survey: 5-Year Data [2014-2018, Block Groups & Larger Areas]
#nhgis extract cleaned
df_income = pd.read_csv('nhgis0005_ds239_20185_2018_zcta.csv')
##nhgis extract cleaned
df_pov= pd.read_csv('nhgis0007_ds239_20185_2018_zcta_pov_individ.csv')
#nhgis extract cleaned
df_race=pd.read_excel('nhgis0006_ds239_20185_2018_zcta_ETHNICITY.xlsx')


#https://www.udsmapper.org/zcta-crosswalk.cfm
df_zip = pd.read_excel('zip_to_zcta_2019.xlsx')

###Also using uszipcode dataset that utilizes super clean API, to add additional fields and calculate possible income for NULL census median income


Create dfs and filter columns

In [4]:
df_income.head()
df_income = df_income[['ZCTA5A','AJZAE001','AJZAM001']]
df_income = df_income.rename(columns={'ZCTA5A':'ZCTA', 'AJZAE001':'Median_Income', 'AJZAM001': 'Margin_of_Error'})

Check shape

In [5]:
df_pov.shape

(33120, 12)

In [6]:
df_income.shape

(33120, 3)

Merge Income and Poverty data

In [7]:
df_income_merge = df_income.merge(df_pov,on='ZCTA', how='left')

Merge UDS Mapper ZCTA crosswalk

In [8]:
df_tx = df_zip.loc[df_zip.STATE=='TX'] 
df_tx['ZCTA']= df_tx['ZCTA'].astype('int64')
df_final= df_income_merge.merge(df_tx, on ='ZCTA', how='inner')


Check Nulls

In [9]:
df_final.isnull().sum()

ZCTA                                                        0
Median_Income                                             157
Margin_of_Error                                           157
 Pov/Income Total                                           0
Under .50                                                   0
.50 to .99                                                  0
1.00 to 1.24                                                0
1.25 to 1.49                                                0
1.50 to 1.84                                                0
1.85 to 1.99                                                0
2.00 and over                                               0
Pov line Total                                              0
Income in the past 12 months below poverty level            0
Income in the past 12 months at or above poverty level      0
ZIP_CODE                                                    0
PO_NAME                                                     0
STATE   

Merge race population information

In [10]:
df_final=df_final.merge(df_race, on='ZCTA', how='left')

Create % totals for ease of use

In [11]:
df_final['Black Percentage of Population'] = df_final['Black or African American'] /   df_final['Total']
df_final['White Percentage of Population'] = df_final['White'] /   df_final['Total']
df_final['American Indian and Alaska Native Percentage of Population'] = df_final['American Indian and Alaska Native'] /   df_final['Total']
df_final['Asian Percentage of Population'] = df_final['Asian'] /   df_final['Total']
df_final['Native Hawaiian and Other Pacific Islander Percentage of Population'] = df_final['Native Hawaiian and Other Pacific Islander'] /   df_final['Total']
df_final['Hispanic or Latino Percentage of Population'] = df_final['Hispanic or Latino'] /   df_final['Total']
df_final['Other Percentage of Population'] = df_final['Other'] /   df_final['Total']



Check Nulls again

In [12]:
df_final.isnull().sum()

ZCTA                                                                     0
Median_Income                                                          157
Margin_of_Error                                                        157
 Pov/Income Total                                                        0
Under .50                                                                0
.50 to .99                                                               0
1.00 to 1.24                                                             0
1.25 to 1.49                                                             0
1.50 to 1.84                                                             0
1.85 to 1.99                                                             0
2.00 and over                                                            0
Pov line Total                                                           0
Income in the past 12 months below poverty level                         0
Income in the past 12 mon

Add some additional information using uszipcode super clean API. This works off of ZCTA as well. Their census data doesn't appear to be as up-to-date as NHGIS

In [13]:
from uszipcode import SearchEngine, SimpleZipcode, Zipcode
county = []
major_city = []
west_bound = []
north_bound = []
south_bound = []
east_bound = []
radius_in_miles = []
land_area_in_sqmi = []
water_area_in_sqmi = []
border_polygon = []

for i in df_final['ZCTA']:
    search = SearchEngine()
    zipcode = search.by_zipcode(i)
    
    county.append(zipcode.county)
    major_city.append(zipcode.major_city)
    west_bound.append(zipcode.bounds_west)
    east_bound.append(zipcode.bounds_east)
    north_bound.append(zipcode.bounds_north)
    south_bound.append(zipcode.bounds_south)
    radius_in_miles.append(zipcode.radius_in_miles)
    land_area_in_sqmi.append(zipcode.land_area_in_sqmi)
    water_area_in_sqmi.append(zipcode.water_area_in_sqmi)


    ##median_income_python.append(zipcode.median_household_income)
df_final['County'] = county
df_final['Major City']= major_city
df_final['West_Bound'] = west_bound
df_final['East_Bound'] = east_bound
df_final['North_Bound'] = north_bound
df_final['South_bound'] = south_bound
df_final['Radius in Miles'] = radius_in_miles    
df_final['Land Area in Sq Mi'] = land_area_in_sqmi    
df_final['Water Area in Sq Mi'] = water_area_in_sqmi 


    


Check Nulls again

In [14]:
df_final.isnull().sum()

ZCTA                                                                     0
Median_Income                                                          157
Margin_of_Error                                                        157
 Pov/Income Total                                                        0
Under .50                                                                0
.50 to .99                                                               0
1.00 to 1.24                                                             0
1.25 to 1.49                                                             0
1.50 to 1.84                                                             0
1.85 to 1.99                                                             0
2.00 and over                                                            0
Pov line Total                                                           0
Income in the past 12 months below poverty level                         0
Income in the past 12 mon

In [15]:
search = SearchEngine()
zipcode = search.by_zipcode(78759)
zipcode.lng

-97.75

Identify Majority Populations by ZCTA

In [17]:

### Get Majority Population for ZCTA

df_final['Majority Pop']= df_final[['Black Percentage of Population','White Percentage of Population',
'American Indian and Alaska Native Percentage of Population','Asian Percentage of Population','Native Hawaiian and Other Pacific Islander Percentage of Population',
'Hispanic or Latino Percentage of Population', 'Other Percentage of Population']].idxmax(axis=1)
##format
df_final['Majority Pop'] = df_final['Majority Pop'].str.replace("Percentage of Population", "")


###Get % under Poverty Line for ZCTA

df_final['Under Poverty Line'] = df_final['Income in the past 12 months below poverty level']/df_final['Pov line Total']
df_final['% Under Poverty Line'] = round(df_final['Under Poverty Line'],2)
df_final['% Under Poverty Line'] = df_final['% Under Poverty Line'].astype(str)+'%'




In [18]:
df_final.columns

Index(['ZCTA', 'Median_Income', 'Margin_of_Error', ' Pov/Income Total',
       'Under .50', '.50 to .99', '1.00 to 1.24', '1.25 to 1.49',
       '1.50 to 1.84', '1.85 to 1.99', '2.00 and over', 'Pov line Total',
       'Income in the past 12 months below poverty level',
       'Income in the past 12 months at or above poverty level', 'ZIP_CODE',
       'PO_NAME', 'STATE', 'ZIP_TYPE', 'zip_join_type', 'Total', 'White',
       'Black or African American', 'American Indian and Alaska Native',
       'Asian', 'Native Hawaiian and Other Pacific Islander', 'Other',
       'Hispanic or Latino', 'Black Percentage of Population',
       'White Percentage of Population',
       'American Indian and Alaska Native Percentage of Population',
       'Asian Percentage of Population',
       'Native Hawaiian and Other Pacific Islander Percentage of Population',
       'Hispanic or Latino Percentage of Population',
       'Other Percentage of Population', 'County', 'Major City', 'West_Bound',
       'E

Let's make a Austin area map showing % Under Poverty line by ZCTA

In [21]:
##regroup and redefine columns that were dropped (df_final will be used for ZIP Crosswalk for use in SQL)
test = df_final.groupby(['ZCTA','Median_Income'], as_index=False).mean()

test = df_final.loc[df_final['Major City']=='Austin']

test['Majority Pop']= test[['Black Percentage of Population','White Percentage of Population',
'American Indian and Alaska Native Percentage of Population','Asian Percentage of Population','Native Hawaiian and Other Pacific Islander Percentage of Population',
'Hispanic or Latino Percentage of Population', 'Other Percentage of Population']].idxmax(axis=1)
##format
test['Majority Pop'] = test['Majority Pop'].str.replace("Percentage of Population", "")

test['% Under Poverty Line'] = round(test['Under Poverty Line'],2)
test['% Under Poverty Line'] = test['% Under Poverty Line'].astype(str)+'%'

##Let's add in the lat long so we can set our folium map without manually typing in lat/long
latitude =[]
longitude =[]
for i in test['ZCTA']:
    search = SearchEngine()
    zipcode = search.by_zipcode(i)
    latitude.append(zipcode.lat)
    longitude.append(zipcode.lng)

test['lat'] = latitude
test['lng'] = longitude


#read in geojson for Texas
geojson= gpd.read_file("tx_texas_zip_codes_geo.geojson")




Check data types

In [23]:
test.dtypes

ZCTA                                                                     int64
Median_Income                                                          float64
Margin_of_Error                                                        float64
 Pov/Income Total                                                      float64
Under .50                                                              float64
.50 to .99                                                             float64
1.00 to 1.24                                                           float64
1.25 to 1.49                                                           float64
1.50 to 1.84                                                           float64
1.85 to 1.99                                                           float64
2.00 and over                                                          float64
Pov line Total                                                         float64
Income in the past 12 months below poverty level    

In [24]:
#Convert to str to join on
test['ZCTA']=test['ZCTA'].astype(str)

In [25]:
#merge to geojson (make sure geojson is on the left-side)
geojson = geojson.merge(test, left_on='ZCTA5CE10',right_on='ZCTA',how='inner')

Check geojson data

In [26]:
geojson.head()

,STATEFP10,ZCTA5CE10,GEOID10,CLASSFP10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,...,North_Bound,South_bound,Radius in Miles,Land Area in Sq Mi,Water Area in Sq Mi,Majority Pop,Under Poverty Line,% Under Poverty Line,lat,lng
0,48,78728,4878728,B5,G6350,S,21002465,47256,+30.4526741,-097.6893683,...,30.489391,30.424418,2.0,8.11,0.02,White,0.115050,0.12%,30.45,-97.69
1,48,78739,4878739,B5,G6350,S,29613883,0,+30.1784193,-097.8886618,...,30.209895,30.139515,3.0,11.43,0.00,White,0.031832,0.03%,30.17,-97.89
2,48,78759,4878759,B5,G6350,S,36045451,0,+30.4026543,-097.7610645,...,30.434275,30.361297,3.0,13.92,0.00,White,0.057781,0.06%,30.40,-97.75
3,48,78759,4878759,B5,G6350,S,36045451,0,+30.4026543,-097.7610645,...,30.434275,30.361297,3.0,13.92,0.00,White,0.057781,0.06%,30.40,-97.75
4,48,78703,4878703,B5,G6350,S,14451214,722351,+30.2932682,-097.7660504,...,30.328108,30.264975,2.0,5.58,0.28,White,0.056416,0.06%,30.29,-97.77


Create Choropleth Map using Choropleth Class (this class is kind of meh)

In [44]:

#set centroids, basemap, and zoom
m = folium.Map(location= [test.lat.mean(),test.lng.mean()], tiles='Stamen Toner', zoom_start=10)

#create choropleth
folium.Choropleth(geo_data=geojson, data=test, columns=[ 'ZCTA','Under Poverty Line'], 
                 name='Choropleth',
                 key_on='feature.properties.ZCTA5CE10', 
                 fill_color='Reds',
                fill_opacity=.7,
                 ).geojson.add_child(folium.GeoJsonTooltip(['ZCTA','Median_Income','Majority Pop','% Under Poverty Line'])).add_to(m) #this took a little bit of tweaking


m
#comment in to save to html(may want to limit the data to areas of interest))
#m.save('Pov_line_Race_Median_Income_ZTCA.html')



Hey it worked! 

Remember all of this can be accomplished with the uszipcode at https://pypi.org/project/uszipcode/. I just opted to use NHGIS extracts for the majority of this work because the Cesus data appears to be more current. Remember again, that this is all ZCTA's, if you want to create crosswalks for your means, I definitely suggest using UDSmapper.



Create file for SQL Crosswalk table

In [ ]:

df_final.to_excel("Income_Race_ZCTA.xlsx",index=False)  